# Sparse-Matrix-Dense-Vector multiplication

## To multiply a sparse two-dimensional mxn matrix (containing considerable number of zeros) with an nx1 dense vector using parallel implementation of segmented reduce and avoiding insignificant calculations

In [1]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
Pkg.add("StaticArrays")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [be33ccc6] + CUDAnative v2.2.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c5f51814] + CUDAdrv v3.0.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [90137ffa] + StaticArrays v0.11.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed BenchmarkTools ─ v0.4.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
 

In [0]:
using CUDAnative, CUDAdrv, CuArrays, StaticArrays, BenchmarkTools, Test

In [3]:
# Function to multiply a matrix and vector by traditional serial method

function sequential_mult(matrix,vector)
  return matrix * vector
end

sequential_mult (generic function with 1 method)

In [4]:
# Kernel to multiply a matrix and vector by parallel implementation of segmented reduce

function SpMv(d_value,d_colidx,d_size,d_maxelement,d_out,d_vector,d_offset)

  idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x
  
  # Bounds check
  if(idx <= size(d_value,1))
  
    d_out[idx] = d_vector[d_colidx[idx]]
    d_out[idx] = d_out[idx] * d_value[idx]
    sync_threads()
    
    # Segmented reduce
    step = 1
    
    while step < d_maxelement
    
      if(((idx-d_offset[idx]) % (step * 2) == 1) && (idx + step) <= d_size[idx])
        @inbounds d_out[idx] += d_out[idx + step]
      end
      
      step *= 2
      
      sync_threads()
    end  
  
  end

return
end

SpMv (generic function with 1 method)

In [16]:
# Read data from file

file = open("/data.txt","r")

IOStream(<file /data.txt>)

In [17]:
str = read(file,String)

"120000450455005080701547"

In [0]:
# Parsing in form of matrix and vector

rows = 5
columns = 4
matrix = zeros(Int,rows,columns)
vector = zeros(Int,columns)
j = 0
for i in str
  j += 1
  if(j <= rows*columns)
    if(i=='"')
      continue
    else
      matrix[j] = parse(Int,i)
    end
  else
    if(i=='"')
      continue
    else
      vector[j - rows*columns] = parse(Int,i)
    end
  end
end

In [22]:
matrix

5×4 Array{Int64,2}:
 1  0  5  0
 2  4  5  8
 0  5  0  0
 0  0  0  7
 0  4  5  0

In [23]:
vector

4-element Array{Int64,1}:
 1
 5
 4
 7

In [0]:
# Array containing non-zero matrix values

h_value = []
for i=1:size(matrix,1)*size(matrix,2)
  if(matrix'[i]!=0)
    push!(h_value,matrix'[i])
  end
end

In [25]:
h_value

10-element Array{Any,1}:
 1
 5
 2
 4
 5
 8
 5
 7
 4
 5

In [0]:
# Array containing column index of each non-zero matrix value

h_colidx = []
for i=1:size(matrix,1)*size(matrix,2)
  if(matrix'[:][i]!=0)
    element = i - size(matrix,2) * div(i,size(matrix,2))
    if(element!=0)
      push!(h_colidx,element)
    else
      push!(h_colidx,size(matrix,2))
    end
  end
end

In [27]:
h_colidx

10-element Array{Any,1}:
 1
 3
 1
 2
 3
 4
 2
 4
 2
 3

In [0]:
# Array containing index(in h_value) of first non-zero element in matrix

h_rowptr = []
count = 0
for i=1:size(matrix,1)
  flag = 0
  for j=1:size(matrix,2)
    if(matrix[i,:][j] != 0)
      count += 1
    end
    if(matrix[i,:][j] != 0 && flag == 0)
      push!(h_rowptr,count)
      flag = 1
    end
  end
end

In [29]:
h_rowptr

5-element Array{Any,1}:
 1
 3
 7
 8
 9

In [0]:
# Array contining cumulative indices for each segment

h_size = []
k = 0
for i=1:size(h_rowptr,1)
  if(i!=size(h_rowptr,1))
    element = h_rowptr[i+1]-h_rowptr[i]
    k = element + k
    for j=1:element
      push!(h_size,k)
    end
  else
    element = size(h_value,1)+1-h_rowptr[i]
    k = element + k
    for j=1:element
      push!(h_size,k)
    end
  end
end

In [31]:
h_size

10-element Array{Any,1}:
  2
  2
  6
  6
  6
  6
  7
  8
 10
 10

In [0]:
# Array contining intermediate result and calculating maximum segment size

inter = []
for i=1:size(h_rowptr,1)
  push!(inter,h_rowptr[i]-1)
end
push!(inter,size(h_value,1))
h_maxelement = -1
for i=1:size(h_rowptr,1)
  if(inter[i+1]-inter[i] > h_maxelement)
    h_maxelement = inter[i+1]-inter[i]
  end
end

In [0]:
# Array containing offsets to the main index in kernel

h_offset = []
for i=1:size(inter,1)-1
  for j=1:inter[i+1]-inter[i]
    push!(h_offset,inter[i])
  end
end

In [42]:
# Array to store result

h_out = zeros(Int,size(h_value,1))

10-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [36]:
# Allocating memory and copying from host to device

d_value = CuArray(Int.(h_value))
d_colidx = CuArray(Int.(h_colidx))
d_rowptr = CuArray(Int.(h_rowptr))
d_size = CuArray(Int.(h_size))
d_out = CuArray(Int.(h_out))
d_vector = CuArray(vector)
d_offset = CuArray(Int.(h_offset))
d_maxelement = h_maxelement

4

In [0]:
@cuda threads = 2^9 SpMv(d_value,d_colidx,d_size,d_maxelement,d_out,d_vector,d_offset)

In [39]:
# h_ans contains the final vector obtained

h_out = Array(d_out)
h_ans = []
for i in h_rowptr
  push!(h_ans,h_out[i])
end
h_ans = Int.(h_ans)

5-element Array{Int64,1}:
 21
 98
 25
 49
 40

In [40]:
sequential_result = sequential_mult(matrix,vector)

5-element Array{Int64,1}:
 21
 98
 25
 49
 40

In [41]:
@test isapprox(sequential_result,h_ans)

Test Passed